In [1]:
import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=RRuntimeWarning)
import sys
import os

In [2]:
import pandas as pd
import numpy as np

In [3]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [4]:
# Enable R magic
%load_ext rpy2.ipython

In [5]:
HERE = os.path.dirname(os.path.realpath('__file__'))
PROJECT = os.path.abspath(os.path.join(HERE, '..'))
DATA = os.path.join(PROJECT, 'data')

print(DATA)

ADPD_SNPS_DATA = os.path.join(DATA, 'adpd_snp_mat.RData')
ADPD_AUTOENCODER_MODELS = os.path.join(DATA, "models")
ADPD_AUTOENCODER_MODEL_STORE = os.path.join(DATA, 'adpd_autoencoder_model_store.RData')
AUTOENCODER_TRAINED_MATRIX = os.path.join(DATA, "autoencoder_trained_matrix.RData")
TRAINED_PATIENT_CLUSTERS = os.path.join(DATA, "trained_patient_clusters.RData")

USER_DATA = os.path.join(DATA, 'Asif_Genotype_Disease_Only_ROSMAP.csv')
SUGBRAPH_SNPS = os.path.join(DATA, 'subgraphs15_snps_mod1.csv')
SUBGRAPH_15_RDATA = os.path.join(DATA, 'subgraph15_snpset148.RData')

INPUT_FOR_MODEL = os.path.join(DATA, "rosmap148.snp.mat.RData")
CLUSTER_CLASSIFIER = os.path.join(DATA, "GLM_model_R")

PREDICTED_USERDATA = os.path.join(DATA, "predicted_userdata.RData")
PREDICTED_USER_Clusters = os.path.join(DATA, "predicted_user_clusters.RData")

/Users/danieldomingo/PycharmProjects/predictme/data


In [6]:
%%R

library(h2o)
library(dplyr)
library(doParallel)

R[write to console]: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


R[write to console]: 
Attache Paket: ‘h2o’


R[write to console]: The following objects are masked from ‘package:stats’:

    cor, sd, var


R[write to console]: The following objects are masked from ‘package:base’:

    %*%, %in%, &&, apply, as.factor, as.numeric, colnames, colnames<-,
    ifelse, is.character, is.factor, is.numeric, log, log10, log1p,
    log2, round, signif, trunc, ||


R[write to console]: 
Attache Paket: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

What is this doing?

In [7]:
%%R
#### main function ####

ann_fun = function(input_data){
  if(dim(input_data)[2] > 1)
  {
    h2o.init(nthreads = -1, max_mem_size = "12g")
    
    #sc = scale(input_data)
    myx = as.h2o(scale(input_data)) 
    
    n = round(dim(input_data)[2])
    m = round(dim(input_data)[2]/2)
    
    hyper_params <- list(hidden = list(1, c(n, 1), c(n,m,1), c(m,1)),
                         input_dropout_ratio = c(0,0.05, 0.2,0.5),
                         l2=10^c(-4:4))
    
    
    r = sample(20:60000, 1)
 
  grid = h2o.grid("deeplearning", 
                    grid_id = paste("mygrid", r, sep="_"), 
                    autoencoder= TRUE,
                  x = colnames(myx),
                    training_frame = myx, 
                    seed=1234567, 
                    stopping_metric="MSE", 
                    stopping_rounds = 5,
                    #activation= "TanhWithDropout",
                    activation= "Tanh",
                    standardize=TRUE,
                    epochs=500,
                    hyper_params = hyper_params)
    
    gbm_sorted_grid <- h2o.getGrid(grid_id = paste("mygrid", r, sep="_"), sort_by = "mse")
    fit <- h2o.getModel(gbm_sorted_grid@model_ids[[1]])
    #nlayers = length(strsplit(gbm_sorted_grid@summary_table[1,1], ",")[[1]])
    #browser()
    nlayers = length(strsplit(substr(gbm_sorted_grid@summary_table[1,1], 2, nchar(gbm_sorted_grid@summary_table[1,1])-1), ",")[[1]])
    newvar = as.data.frame(h2o.deepfeatures(fit, myx, nlayers))
 
  
    #Rename column
 
    newvar = as.data.frame(newvar[,1])
    colnames(newvar) = paste( sub("_.*", '', colnames(input_data)[1]) , sub(".*_", '',colnames(input_data)[1]) , sep= "_")
    
   
    #save meta-features in variable 
    subgraph_feature = data.frame(matrix(NA, nrow = 844, ncol = 1)) # nrow = Patient number
    colnames(subgraph_feature) = "dummy"
 
  
    subgraph_feature = cbind(subgraph_feature,newvar) 
    subgraph_feature$dummy = NULL
    
  }else if(dim(input_data)[2] == 1 ){
    subgraph_feature = as.data.frame(input_data)
    colnames(newvar) = paste( sub("_.*", '', colnames(input_data)[1]) , sub(".*_", '',colnames(input_data)[1]) , sep= "_")
  }else{
    subgraph_feature = data.frame(matrix(NA, nrow = 844, ncol = 1)) # nrow = Patient number
    colnames(subgraph_feature) = "dummy"
  }
  outcome = list(subgraph_feature, fit) # scales=attributes(sc)
  return(outcome)
}

In [ ]:
%%R -i=ADPD_SNPS_DATA,ADPD_AUTOENCODER_MODEL,ADPD_SNPS_DATA
# see: https://ipython.org/ipython-doc/2/config/extensions/rmagic.html

load(ADPD_SNPS_DATA)
# apply main function on each mechanisms matrices
model.store <- lapply(snp_mat, function(x) ann_fun(x))
                      
# Stores the model
# save(model.store, file = "model.store.rosmapIdibaps-148_full.RData")




H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /var/folders/yx/ppwbtpyn61l2796mnr1t62f00000gn/T//Rtmp6rhN2K/h2o_danieldomingo_started_from_r.out
    /var/folders/yx/ppwbtpyn61l2796mnr1t62f00000gn/T//Rtmp6rhN2K/h2o_danieldomingo_started_from_r.err


Starting H2O JVM and connecting: ... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         5 seconds 432 milliseconds 
    H2O cluster timezone:       Europe/Berlin 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.28.0.2 
    H2O cluster version age:    6 days  
    H2O cluster name:           H2O_started_from_R_danieldomingo_iwe304 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   10.66 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:  

In [ ]:
save(model.store, file = "new-model.store.rosmapIdibaps-148_full.RData")


In [ ]:
%%R
adpd_autoencoder_matrix=matrix(nrow = 844, ncol = 15) # 358 PD patients and 15 subgraphs

j=1
for (i in model.store){
  autoen[,j]=i[[1]][[1]]
  j=j+1
}
rownames(autoen) <- rownames(snp_mat[[1]])
colnames(autoen) <- names(model.store)
save(adpd_autoencoder_matrix, file = ADPD_AUTOENCODER_MATRIX)

#~~~~~: save models in h2o format :~~~~~~#

an_model <- lapply(model.store, '[[',2)  # This returns a list with only the 2nd element (h2o models)

for (i in an_model){
  h2o.saveModel(i, path = ADPD_AUTOENCODER_MODEL)
}                     

In [19]:
%%R -i=SUBGRAPH_15_RDATA,USER_DATA,INPUT_FOR_MODEL

load(SUBGRAPH_15_RDATA)
userSNPs <- read.csv(USER_DATA, row.names = 1, stringsAsFactors=FALSE)
userSNPs <- as.data.frame(t(userSNPs), stringsAsFactors = FALSE)

user_snp_mat = list()
k = 1
for (i in subgraph15.snps){
  user_snp_mat[[k]] = select(userSNPs, i)
  k = k + 1
}

names(user_snp_mat) <- names(subgraph15.snps)
save(user_snp_mat,file = INPUT_FOR_MODEL)


## Predict User data matrix based on our Training Dataset autoencoder models

In [10]:
%%R -i=ADPD_AUTOENCODER_MODEL_STORE,ADPD_AUTOENCODER_MODELS,INPUT_FOR_MODEL,PREDICTED_USERDATA
#~~~~~: Predict User data matrix based on our Training dataset autoencoder models :~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

#~~~~~~~~~~: load trained autoencoder matrix :~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
load(ADPD_AUTOENCODER_MODEL_STORE)

#~~~~~~~~~~: load trained autoencoder models :~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

h2o.init()
setwd(ADPD_AUTOENCODER_MODELS)
modelnames <-list.files(path = ".")
models = lapply(paste0(getwd(),"/", modelnames), h2o.loadModel)

#~~~~~~~~~~: load User files :~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

#### all subgraphs (samples * SNPs coding matrix each) into an RData file
testmat <- INPUT_FOR_MODEL
testh2o <- sapply(1:length(testmat), function(x) as.h2o(testmat[[x]]))


#~~~~~~~~~~~~~: Mechanism Score Prediction :~~~~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

predicted_userdata <- foreach(i = seq(model.store)) %do% {
  if (length(attributes(models[[model.store[[i]][[2]]@model_id]])$parameters$hidden) == 1) {
    tmp = as.data.frame(h2o.deepfeatures(models[[model.store[[i]][[2]]@model_id]], testh2o[[i]], 1))
  } else if (length(attributes(models[[model.store[[i]][[2]]@model_id]])$parameters$hidden) > 1){
    tmp = as.data.frame(h2o.deepfeatures(models[[model.store[[i]][[2]]@model_id]], testh2o[[i]], 2))
    tmp = cbind(tmp)
  }
}

print(predicted_userdata)
predicted_userdata <- do.call(cbind, pred)
names(predicted_userdata) <- names(model.store)
rownames(predicted_userdata) <- rownames(userSNPs)
save(predicted_userdata,file = PREDICTED_USERDATA)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         58 minutes 3 seconds 
    H2O cluster timezone:       Europe/Berlin 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.2 
    H2O cluster version age:    6 months !!! 
    H2O cluster name:           H2O_started_from_R_danieldomingo_duc237 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   5.68 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.6.2 (2019-12-12) 

  |======================================================================| 100%
[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL


R[write to console]: Fehler in names(predicted_userdata) <- names(model.store) : 
  se intenta especificar un atributo en un NULL
Ruft auf: <Anonymous> -> <Anonymous> -> withVisible

R[write to console]: Además: 
R[write to console]: Warnmeldung:

R[write to console]: In h2o.clusterInfo() :
R[write to console]:  
Your H2O cluster version is too old (6 months)!
Please download and install the latest version from http://h2o.ai/download/




Fehler in names(predicted_userdata) <- names(model.store) : 
  se intenta especificar un atributo en un NULL
Ruft auf: <Anonymous> -> <Anonymous> -> withVisible


## Predict Clusters for User Data based on the Classifier trained with Training Dataset 

In [16]:
%%R -i=AUTOENCODER_TRAINED_MATRIX,TRAINED_PATIENT_CLUSTERS,CLUSTER_CLASSIFIER

### Start up a 1-node H2O server on the local machine, and allow it to use all CPU cores and up to 6GB of memory:
h2o.init(nthreads=-1, min_mem_size="6G")

### Import autoencoder data set #subgraph15
load(AUTOENCODER_TRAINED_MATRIX)
main_data <- data.frame(autoen)

#### read the cluster assignments of each patients
load(TRAINED_PATIENT_CLUSTERS)
rownames(clusters) <- clusters[,1]
clusters[,1] <- NULL


##### merge cluster assignment to the dataset
fin_data <- merge(clusters,main_data,by="row.names")
fin_data$clusters <- as.factor(fin_data$clusters)
rownames(fin_data) = fin_data$Row.names
fin_data <- fin_data[,-1]
fullD <- as.h2o(fin_data) # get complete data set into h2o frame for cross validation approach

##### Training Classifier

y = "clusters" # response variable
#x = names(trainData)
x = names(fullD)
x = x[-which(x==y)] # predictor variables

### Train Model
snpModel = h2o.glm(training_frame = fullD, 
                   #training_frame = trainD, # keep it commented while using cross validation
                   #validation_frame = validD, # keep it commented while using cross validation
                   x = x, 
                   y = y,
                   nfolds = 10, 
                   family='multinomial',
                   solver='L_BFGS',
                   lambda_search=TRUE)

print(snpModel)
h2o.performance(snpModel, xval = TRUE)
h2o.saveModel(snpModel, path=CLUSTER_CLASSIFIER)

#### Now predict the Test Data with the trained model #####
#snpModel = h2o.loadModel("/home/memon/genetic_analyses/ann/output/subgraph15/classifier_adpd_ann30/GLM_model_R_1552061363922_1")


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /var/folders/yx/ppwbtpyn61l2796mnr1t62f00000gn/T//RtmpwXjRnz/h2o_danieldomingo_started_from_r.out
    /var/folders/yx/ppwbtpyn61l2796mnr1t62f00000gn/T//RtmpwXjRnz/h2o_danieldomingo_started_from_r.err


Starting H2O JVM and connecting: . Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 seconds 242 milliseconds 
    H2O cluster timezone:       Europe/Berlin 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.2 
    H2O cluster version age:    6 months !!! 
    H2O cluster name:           H2O_started_from_R_danieldomingo_duc237 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   5.75 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:

R[write to console]: Fehler in readChar(con, 5L, useBytes = TRUE) : 
  no se puede abrir la conexión
Ruft auf: <Anonymous> -> <Anonymous> -> withVisible -> load -> readChar

R[write to console]: Además: 
R[write to console]: Warnmeldungen:

R[write to console]: 1: 
R[write to console]: In h2o.clusterInfo() :
R[write to console]:  
Your H2O cluster version is too old (6 months)!
Please download and install the latest version from http://h2o.ai/download/

R[write to console]: 2: 
R[write to console]: In readChar(con, 5L, useBytes = TRUE) :
R[write to console]: 
 
R[write to console]:  cannot open file '/Users/danieldomingo/PycharmProjects/predictme/data/models': it is a directory




Fehler in readChar(con, 5L, useBytes = TRUE) : 
  no se puede abrir la conexión
Ruft auf: <Anonymous> -> <Anonymous> -> withVisible -> load -> readChar


In [ ]:
%%R -i=CLUSTER_CLASSIFIER

#### Now predict the Test Data with the trained model #####
snpModel = h2o.loadModel(CLUSTER_CLASSIFIER)

#### for autoencoder predcited AET_PD data based predcition ####
load(PREDICTED_USERDATA) # rosmap dataset predcited (h2o.deepfeature) with autoen30 model
testh2o <- as.h2o(predicted_userdata)

##########################################

### prediction on test data set
prediction = h2o.predict(snpModel, newdata=testh2o)

predicted.cl = as.data.frame(prediction$predict)
names(predicted.cl) <- "clusters"
predicted.cl$clusters <- as.integer(as.character(gsub("Cluster_", "", predicted.cl$clusters)))

# merge predicted clusters to test data
# so all patients in test data set get a cluster assignment based on their mechanism profile (predictro variables)
predicted_cl.testdata <- cbind(predicted.cl,testdata)
#save(predicted_cl.testdata,file="/home/memon/genetic_analyses/ann/output/subgraph15/ann30_cluster_prediction_adni148_nonad.RData")


In [ ]:
%%R 

autoen